# Convolution Image Net

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.nn_visualization import variable_summaries
from utils.nn_graph import simple_layer
from utils.data import init_model_logging
from utils.nn_visualization import init_embedding_projector, init_embedding_data
from utils.nn_visualization import get_sprite_img, get_label_class_names
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import pandas as pd
import matplotlib.pylab as plt

## Load data

In [ ]:
data = input_data.read_data_sets('/data/fashion/', one_hot=True)
img_shape = (28, 28)
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

## Build Conv Net Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('conv_image_net_inputs'):
        images = tf.placeholder(tf.float32, shape=[None, 784], name='images')
        labels = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
        keep_dropout_prob = tf.placeholder(tf.float32, name='keep_dropout_prob')
        is_training = tf.placeholder(tf.bool, name='is_training')
            
    with tf.name_scope('image_reshape'):        
        images_reshaped = tf.reshape(images, [-1, 28, 28, 1])

    with tf.variable_scope('conv_layer_1'):
        pass
        #######################################################
        # conv layer, 32 filters (3, 3), stride (1, 1), SAME  #
        # bias, relu                                          #
        # max pooling, (2, 2), stride (2, 2), SAME            # 
        #######################################################

    with tf.variable_scope('conv_layer_2'):
        pass
        #######################################################
        # conv layer, 64 filters (3, 3), stride (1, 1), SAME  #
        # bias, relu                                          #
        # max pooling, (2, 2), stride (2, 2), SAME            # 
        #######################################################
    
    with tf.variable_scope('feed_forward_layer_1'):
        raw_prediction = tf.reshape(images_reshaped, [-1, 28 * 28])
        raw_prediction = simple_layer('ff_1', raw_prediction, shape=[28 * 28, 10], activation='linear')
        raw_prediction = tf.nn.dropout(raw_prediction, keep_dropout_prob)

    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(raw_prediction)
        print('Debug [prediction]:', prediction.get_shape())
    
    with tf.variable_scope("embedding_visualization"):
        embedding = tf.Variable(tf.zeros([5000, 10]), name='valid_embedding')
        embedding_assignment = embedding.assign(raw_prediction)

    with tf.name_scope('loss'):
        cross_entropy_vector = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=raw_prediction)
        loss = tf.reduce_mean(cross_entropy_vector)
        variable_summaries('loss_summary', cross_entropy_vector)
        
    with tf.name_scope('training'):
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_step  = tf.train.AdamOptimizer().minimize(loss)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        variable_summaries('accuracy_summary', correct_prediction)
    
    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

## Init Model Logging

In [ ]:
base_dir = '/tensorboard_summaries_mnt/conv_image_net/'
logging_meta = init_model_logging(base_dir, 'experiment_1', graph=graph, remove_existing=True)

## Add Embedding Projection

In [ ]:
sprite_img = get_sprite_img(data.validation.images, img_shape)
label_names = get_label_class_names(data.validation.labels, class_id2class_name_mapping)

init_embedding_data(logging_meta['valid_writer_dir'], sprite_img, label_names)
init_embedding_projector(logging_meta['valid_writer'], embedding, img_shape)

## Debug Conv Net

In [ ]:
session = tf.Session(graph=graph)
session.run(initialize_vars)

In [ ]:
_images, _labels = data.train.next_batch(10)
feed_dict = {images: _images, labels: _labels, keep_dropout_prob: 1., is_training: False}
print("Debug [cross_entropy_vector]:", session.run(tf.shape(images_reshaped), feed_dict))

In [ ]:
session.close()

## Run Conv Net

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']

validation_feed_dict = {
    images: data.validation.images, 
    labels: data.validation.labels,
    keep_dropout_prob: 1., 
    is_training: False}

with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    for iteration in range(5001):
        ##################
        # Training phase #
        ##################
        _images, _labels = data.train.next_batch(100)
        feed_dict = {images: _images, labels: _labels, keep_dropout_prob: 0.5, is_training: True}
        _ = session.run([train_step], feed_dict=feed_dict)
        
        if iteration % 10 == 0:
            feed_dict={images: _images, labels: _labels, keep_dropout_prob: 1., is_training: False}
            _summary, _accuracy, _loss = session.run([merge_summaries, accuracy, loss],feed_dict=feed_dict)
            
            logging_meta['train_writer'].add_summary(_summary, iteration)
            print("Train Iteration {}: loss {}, accuracy {}".format(iteration, _loss, _accuracy))
      
        ####################
        # Validation phase #
        ####################
        if iteration % 100 == 0:
            fetches = [embedding_assignment, merge_summaries, accuracy, loss]
            _, _summary, _accuracy, _loss = session.run(fetches, validation_feed_dict)
            
            logging_meta['valid_writer'].add_summary(_summary, iteration)
            logging_meta['saver'].save(session, model_path, iteration)
            print("= Valid Iteration {}: loss {}, accuracy {} =".format(iteration, _loss, _accuracy))
            
    _prediction, = session.run([prediction], feed_dict=validation_feed_dict)

## Results evaluation

In [ ]:
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_accuracy
from utils.results_evaluation import get_false_positives
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_rec_prec
from utils.results_evaluation import plot_coocurance_matrix
from utils.results_evaluation import plot_examples 

In [ ]:
df = get_info_df(data.validation.labels, _prediction, class_id2class_name_mapping, data.validation.images)

In [ ]:
get_accuracy(df)

In [ ]:
get_accuracy(df, True)

In [ ]:
get_rec_prec(df, class_id2class_name_mapping)

In [ ]:
fp = get_false_positives(df, 'Shirt', 'Bag')

In [ ]:
plot_examples(fp)

In [ ]:
plot_coocurance_matrix(df, use_top3=False, use_log=False)